In [319]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from dotenv import load_dotenv
from openai import OpenAI
import json
import ast
import asyncio

os.makedirs('data/cached', exist_ok=True)

# Article Crawling

In [161]:
class ArticleCrawling:

    def __init__(self, url):
        self.url = url
        self.soup = self._get_soup()

    def _get_soup(self):
        response = requests.get(self.url)
        return BeautifulSoup(response.text, 'html.parser')
    
    def get_article_url(self):
        meta_tag = self.soup.find("meta", property = "og:url")
        return meta_tag["content"] if meta_tag else None
    
    def get_id(self):
        url = self.get_article_url()    # https://www.spnews.co.kr/news/articleView.html?idxno=101049

        # source(spnew, ytn 등등)
        source = url.split("//")[1]
        source = source.split("/")[0]
        source = source.replace("www.", "").split(".")[0]

        # id
        if "idxno" not in url:
            return None
        idx = url.split("idxno=")[1]
        return f"{source}_{idx}"
    
    def get_title(self):
        header = self.soup.find("h1", class_ = "heading")
        return header.get_text(strip = True) if header else None
      
    def get_author(self):
        tag = self.soup.find("i", class_ = "icon-user-o")
        if tag:
            author = tag.parent.get_text(strip = True)
            return author.replace("기자명", "").strip()
        
    def get_category(self):
        breadcrumb = self.soup.find("ul", class_ = "breadcrumbs")
        if breadcrumb:
            for a in breadcrumb.find_all("a"):
                text = a.get_text(strip=True)
                if text in ["정치", "외교", "군사", "경제/산업", "사회/문화/체육"]:
                    return text
    
    def get_publish_date(self):
        tag = self.soup.find("i", class_ = "icon-clock-o")
        if tag:
            raw = tag.parent.get_text(strip = True)
            return raw.replace("입력", "").strip()
        
    def get_contents(self):
        article_body = self.soup.find_all("span", style = "font-size:18px;")
        contents = "\n".join(i.get_text(strip = True) for i in article_body)
        # return contents.rstrip("@") 
        return contents.split('@')[0].strip()
    
    def to_dict(self):
        return {
            "url": self.get_article_url(),
            "id": self.get_id(),
            "title": self.get_title(),
            "author": self.get_author(),
            "category": self.get_category(),
            "publish_date": self.get_publish_date(),
            "contents": self.get_contents()
        }
    
    def __str__(self):
        data = self.to_dict()
        return data


In [162]:
url_list = [
    'https://www.spnews.co.kr/news/articleView.html?idxno=101049',
    'https://www.spnews.co.kr/news/articleView.html?idxno=101366',
    'https://www.spnews.co.kr/news/articleView.html?idxno=100345',
    'https://www.spnews.co.kr/news/articleView.html?idxno=101306',
    'https://www.spnews.co.kr/news/articleView.html?idxno=100889',
    'https://www.spnews.co.kr/news/articleView.html?idxno=100318',
    'https://www.spnews.co.kr/news/articleView.html?idxno=101377',
    'https://www.spnews.co.kr/news/articleView.html?idxno=101097',
    'https://www.spnews.co.kr/news/articleView.html?idxno=100356',
    'https://www.spnews.co.kr/news/articleView.html?idxno=101404',
    'https://www.spnews.co.kr/news/articleView.html?idxno=101394',
    'https://www.spnews.co.kr/news/articleView.html?idxno=101258',
    'https://www.spnews.co.kr/news/articleView.html?idxno=101402',
    'https://www.spnews.co.kr/news/articleView.html?idxno=101369',
    'https://www.spnews.co.kr/news/articleView.html?idxno=101272',
    'https://www.spnews.co.kr/news/articleView.html?idxno=101394',
]

In [163]:
def make_test_dataset(url_list):
    test_df = pd.DataFrame(columns=['id', 'title', 'contents', 'author', 'publish_date', 'url', 'category'])

    for url in url_list:
        test1 = ArticleCrawling(url).to_dict()

        # DataFrame에 한 행 추가
        test_df.loc[len(test_df)] = [
            test1['id'],
            test1['title'],
            test1['contents'],
            test1['author'],
            test1['publish_date'],
            test1['url'],
            test1['category'],
        ]

    return test_df.reset_index(drop=True)


In [164]:
test_df = make_test_dataset(url_list)

In [165]:
test_df.head(5)

,id,title,contents,author,publish_date,url,category
0,spnews_101049,"北, 김영남 상임위원장 영결식...김정은 참석",고(故) 김영남 북한 최고인민회의 상임위원회 위원장 장례식이 5일 평양에서 국장으로...,안윤석 대기자,2025.11.06 07:40,https://www.spnews.co.kr/news/articleView.html...,정치
1,spnews_101366,"박태성 北 내각총리, 광산·발전소 등 경제 여러 부문 현장 확인",박태성 북한 내각총리가 광산과 발전소 등 경제 여러 부문 현장을 확인했다.\n노동신...,안윤석 대기자,2025.11.16 06:47,https://www.spnews.co.kr/news/articleView.html...,정치
2,spnews_100345,"김정은, '신의주온실종합농장' 또 현지지도...""농촌문명의 새로운 경지 개척""",김정은 북한 총비서가 17일 마감단계에 들어선 신의주온실종합농장 건설장을 또다시 방...,안윤석 대기자,2025.10.18 07:02,https://www.spnews.co.kr/news/articleView.html...,정치
3,spnews_101306,"박태성 北 내각총리, 통싸완 폼비한 라오스 외무상 접견",박태성 북한 내각총리가 13일 만수대의사당에서 통싸완 폼비한 라오스 외무상을 만났다...,안윤석 대기자,2025.11.14 07:17,https://www.spnews.co.kr/news/articleView.html...,외교
4,spnews_100889,"북한-러시아, 무역경제 및 과학기술협력 추진 회담...의정서 조인",북한과 러시아가 합의된 다방면적인 쌍무협력계획 이행을 추진하기 위한 회담을 31일 ...,안윤석 대기자,2025.11.01 07:15,https://www.spnews.co.kr/news/articleView.html...,외교


In [166]:
# 테스트df 복사본 생성
test_data = test_df.copy()

# 테스트 데이터의 게재일 datetime으로 변환
test_data['publish_date'] = pd.to_datetime(test_data['publish_date']).dt.date
test_data.head(2)

# 테스트 데이터 저장
os.makedirs('data', exist_ok=True)
test_data.to_csv('data/test_dataset.tsv', index=False, sep='\t')

# LLM Summarizer 구현

In [320]:
# .env 파일 로드
load_dotenv()  # 현재 디렉토리 내 .env 파일 정보를 환경변수로 읽어옴

# 환경변수 확인
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found. .env 또는 시스템 환경변수에 설정하세요.")

In [321]:
# OpenAI 클라이언트 초기화
client = OpenAI(api_key=api_key)

In [322]:
def split_to_list(s):
    """
    쉼표로 분리된 단어들로 이루어진 문자열을 단어 리스트로 변환
    """
    if not s:
        return []
    if isinstance(s, list):
        return s
    return [x.strip() for x in s.split(",")]

In [334]:
# 뉴스요약 LLM 함수
def get_article_summary(title, contents, publish_date):
    """
    뉴스 기사를 LLM으로 요약하고 항목별 데이터 반환
    
    Change Log:
        @@ 11.18 
        > 프롬프트 수정
            - 키값 정리 형식 재 설정 - 데이터 프레임의 컬럼명이 키값이 되도록 수정
            - 추가적인 컬럼 event_person 추가
            - event_obj 컬럼명 event_org로 변경
        @@ 11.19
        > 프롬프트 수정
            - 인물명은 이름만 명확하게
            - 지역명은 [국가, 도, 시] 단위로 명확하게
            - 평양 쌀, 옥수수, 달러환율 정보는 각기 별개의 데이터로 저장
            - 키워드는 인물명, 지역명을 반드시 포함 + 요약내용 대표 단어 추가
        > result 파싱 방식 수정
            - eval()에서 jason.load()로 변경
    """
    
    prompt = f"""
    아래 기사를 분석하여 요구된 정보를 작성하시오.

    # 기사 제목:
    {title}

    # 기사 내용:
    {contents}

    # 기사 작성일:
    {publish_date}

    1. 아래 형식으로 정리 (괄호안 각 key값의 한글 설명은 참고만 하고 최종 결과에는 포함하지 않음)
    - summary(주요 사건 요약):
    - event_title(사건 주제):
    - event_date(사건 발생일):
    - event_person(사건 핵심 인물):
    - event_org(사건 핵심 조직/기관):
    - event_loc(사건 발생 지명):
    - keywords(주요 키워드):
    - p_rice(won/kg)(쌀 가격):
    - p_corn(won/kg)(옥수수 가격):
    - p_usd(won/usd)(달러 가격)

    2. 각 카테고리의 조건
    - "summary": 3 문장 이하로 핵심 내용만 발췌.
    - "event_title": 간단한 한 문장으로 사건 주제 작성.
    - "event_date": yyyy-mm-dd 형식, 기사에 "event_date"가 명시되지 않았으면 "기사 내용" 중 시간 또는 기간을 나타내는 단어(예시로, '어제', '사흘전', '일주일 전' 등)를 참고하여 "기사 작성일" 기준 계산.
    - "event_person": 사건의 주체 인물(들)의 이름만 입력, 다수의 경우 쉼표로 구분.
    - "event_org": 사건의 주체 조직 및 기관의 이름만 입력, 다수의 경우 쉼표로 구분, **언론사명은 반드시 제외**, **신문사명은 반드시 제외**, **기자가 참고한 출처의 이름도 반드시 제외**, **"노동신문"은 반드시 제외**.
    - "event_loc": [도, 시]단위 지명만을 입력, 건물등에서 일어난 사건의 경우는 해당 장소의 [국가, 도, 시] 지명을 입력, 행정구역이 "시"일 경우는 꼭 "시"를 명시 (개성시, 평양시, 고성시 등), 특히 "평양" / "평양직할시" / "평양시"와 같이 한 지명에 다양한 표기가 있을경우는 "평양시" ([시 이름] + 시)와 같은 형태로 통일, **"북한" 이라는 단어는 반드시 제외**.
    - "keywords": "summary", "event_title", "event_person", "event_org", "event_loc" 모두를 종합적으로 고려하여 해당 뉴스 사건을 대표할 수 있는 **단어 5개 선정**, **"북한" 이라는 단어는 반드시 제외**, 쉼표로 구분하여 입력.
    - 물가 관련 뉴스에 쌀, 옥수수 kg당 가격 및 원달러 환율이 제공될 경우, 반드시 평양의 정보를 선택하여 각 값을 정수로 입력. 
    - 위 결과를 종합하여 딕셔너리 형태로 출력.
    - 설명 금지, 답만 출력.
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "당신은 북한 관련 뉴스 사건 정보를 추출하는 전문 분석 모델입니다."},
            {"role": "user", "content": prompt},
        ],
        temperature=0
    )

    # 소모 토큰량 추출
    input_tokens = response.usage.prompt_tokens
    output_tokens = response.usage.completion_tokens


    result_text = response.choices[0].message.content.strip()

    # 문자열을 dict로 변환
    try:
        result = json.loads(result_text)  # json 파싱으로 수정(11.19)
    except:
        print("Parsing error:", result_text)
        return None

    return result, input_tokens, output_tokens

In [335]:
def get_summary_df(df, cache_file="cache_output.csv"):
    """
    뉴스 요약 파싱 정보를 DataFrame으로 변환

    Args:
        df (pd.DataFrame): 원본 뉴스 데이터프레임. 최소 컬럼 'id', 'title', 'contents', 'publish_date' 포함.
        cache_file (str, optional): 요약 결과를 저장/로드할 캐시 파일명. Defaults to "cache_output.csv".

    Returns:
        pd.DataFrame: 뉴스 요약 및 사건 정보가 포함된 DataFrame. 
                      컬럼:
                        - 'id': 뉴스 ID
                        - 'summary': 주요 사건 요약 (문자열)
                        - 'keywords': 주요 키워드 (리스트)
                        - 'event_title': 사건 제목 (문자열)
                        - 'event_date': 사건 발생일 (yyyy-mm-dd)
                        - 'event_person': 사건 핵심 인물 (리스트)
                        - 'event_org': 사건 핵심 조직/기관 (리스트)
                        - 'event_loc': 사건 발생 지명 (리스트)
                        - 'p_rice(won/kg)': 평양 쌀 가격 (정수)
                        - 'p_corn(won/kg)': 평양 옥수수 가격 (정수)
                        - 'p_usd(won/usd)': 원/달러 환율 (정수)
                        - 'job_cost': 처리 비용 추정 (float)
    
    Example:
        >>> df_summary = get_article_summary(news_df)
        >>> df_summary.head()

    Change Log:
        @@ 11.19
        - 캐싱 기능 추가: 뉴스 id 기반으로 이미 요약된 뉴스는 스킵하도록 수정.
        - 새로운 뉴스가 추가될 경우에만 LLM 호출 수행.
        - 쉼표로 구분된 항목(event_person, event_org, event_loc, keywords)을 리스트 형태로 변환.
    
    """
    
########## 코드 시작 ####bz######
    # ---------------------------------------------------------------------------------
    # 0. 기본 설정 / 스키마 정의
    # ---------------------------------------------------------------------------------
    new_cols = [
        "summary", "keywords", "event_title", "event_date", "event_person",
        "event_org", "event_loc", "p_rice(won/kg)", "p_corn(won/kg)",
        "p_usd(won/usd)", "job_cost"
    ]

    cache_dir = "data/cached"
    os.makedirs(cache_dir, exist_ok=True)
    cache_file_path = os.path.join(cache_dir, cache_file)

    # 스키마 정의 (dtype 안정화)
    schema = {
        "id": pd.Series(dtype="object"),
        "summary": pd.Series(dtype="object"),
        "keywords": pd.Series(dtype="object"),
        "event_title": pd.Series(dtype="object"),
        "event_date": pd.Series(dtype="object"),
        "event_person": pd.Series(dtype="object"),
        "event_org": pd.Series(dtype="object"),
        "event_loc": pd.Series(dtype="object"),
        "p_rice(won/kg)": pd.Series(dtype="Int64"),
        "p_corn(won/kg)": pd.Series(dtype="Int64"),
        "p_usd(won/usd)": pd.Series(dtype="Int64"),
        "job_cost": pd.Series(dtype="float"),
    }

    # ---------------------------------------------------------------------------------
    # 1. 캐시 로드 or 초기화
    # ---------------------------------------------------------------------------------
    if os.path.exists(cache_file_path):
        print("⏳ 캐시 파일 로드 중...")
        output_df = pd.read_csv(cache_file_path)

        # CSV는 리스트를 문자열로 저장하므로 복원 필요
        list_cols = ["keywords", "event_person", "event_org", "event_loc"]
        for col in list_cols:
            if col in output_df:
                output_df[col] = output_df[col].apply(
                    lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith("[") else x
                )

        # schema 기준으로 dtype 강제 적용
        for col, series in schema.items():
            if col not in output_df.columns:
                output_df[col] = series  # 없는 컬럼은 빈 Series 생성
            else:
                output_df[col] = output_df[col].astype(series.dtype, errors="ignore")

        print("✔ 캐시 반영 완료!")
    else:
        print("캐시 없음 → 빈 DF 생성")
        output_df = pd.DataFrame(schema)

    # ---------------------------------------------------------------------------------
    # 2. 신규 기사만 처리
    # ---------------------------------------------------------------------------------
    cached_ids = set(output_df["id"].astype(str).tolist())
    new_articles = df[~df["id"].astype(str).isin(cached_ids)]

    print(f"신규 샘플 발견: {len(new_articles)}개")

    # ---------------------------------------------------------------------------------
    # 3. 루프 처리 / row 1개씩 추가
    # ---------------------------------------------------------------------------------
    for i, (idx, row) in enumerate(new_articles.iterrows()):
        article_id = row["id"]
        title = row["title"]
        contents = row["contents"]
        publish_date = row["publish_date"]

        print(f"Summarizing [{article_id}] ...")

        # LLM 호출
        result, input_tokens, output_tokens = get_article_summary(title, contents, publish_date)

        # 새 row 구성 (None → pd.NA)
        new_data = {
            "id": article_id,
            "summary": result.get("summary"),
            "keywords": split_to_list(result.get("keywords")),
            "event_title": result.get("event_title"),
            "event_date": result.get("event_date"),
            "event_person": split_to_list(result.get("event_person")),
            "event_org": split_to_list(result.get("event_org")),
            "event_loc": split_to_list(result.get("event_loc")),
            "p_rice(won/kg)": pd.NA if result.get("p_rice(won/kg)")==0 or result.get("p_rice(won/kg)") is None else result.get("p_rice(won/kg)"),
            "p_corn(won/kg)": pd.NA if result.get("p_rice(won/kg)")==0 or result.get("p_corn(won/kg)") is None else result.get("p_corn(won/kg)"),
            "p_usd(won/usd)": pd.NA if result.get("p_usd(won/usd)")==0 or result.get("p_usd(won/usd)") is None else result.get("p_usd(won/usd)"),
            "job_cost": (
                input_tokens * 0.15 / 1_000_000 +
                output_tokens * 0.60 / 1_000_000
            ),
        }

        # 컬럼 순서와 dtype 맞춰서 row 추가
        new_data_aligned = {col: new_data.get(col, pd.NA) for col in output_df.columns}
        
        new_row_df = pd.DataFrame([new_data_aligned]).dropna(axis=1, how='all')  # dict → DataFrame 변환, na값이 있는 컬럼 제거
        output_df = pd.concat([output_df, new_row_df], ignore_index=True)

        # 캐시 저장
        output_df.to_csv(cache_file_path, index=False)

        print(f"Job Complete! [{article_id}] ({i+1}/{len(new_articles)})")

    print("전체 작업 완료!")
    return output_df

In [336]:
df = pd.read_csv('data/spnews_ver2.csv')
input_data = df[df.contents.notnull() & df.category.notnull()]

In [ ]:
summary_df = get_summary_df(input_data)

캐시 없음 → 빈 DF 생성
신규 샘플 발견: 9104개
Summarizing [spnews_101404] ...
Job Complete! [spnews_101404] (1/9104)
Summarizing [spnews_101403] ...
Job Complete! [spnews_101403] (2/9104)
Summarizing [spnews_101402] ...
Job Complete! [spnews_101402] (3/9104)
Summarizing [spnews_101394] ...
Job Complete! [spnews_101394] (4/9104)
Summarizing [spnews_101392] ...
Job Complete! [spnews_101392] (5/9104)
Summarizing [spnews_101391] ...
Job Complete! [spnews_101391] (6/9104)
Summarizing [spnews_101390] ...
Job Complete! [spnews_101390] (7/9104)
Summarizing [spnews_101389] ...
Job Complete! [spnews_101389] (8/9104)
Summarizing [spnews_101383] ...
Job Complete! [spnews_101383] (9/9104)
Summarizing [spnews_101377] ...
Job Complete! [spnews_101377] (10/9104)
Summarizing [spnews_101376] ...
Job Complete! [spnews_101376] (11/9104)
Summarizing [spnews_101374] ...
Job Complete! [spnews_101374] (12/9104)
Summarizing [spnews_101373] ...
Job Complete! [spnews_101373] (13/9104)
Summarizing [spnews_101369] ...
Job Compl

## 테스트 (11.19) - spnews_ver2.csv 결과중 50개 랜덤 추출 후 요약

In [296]:
df = pd.read_csv('data/spnews_ver2.csv')

test_df = pd.concat([
    df.head(13),
    df[df.title.str.contains('물가')].iloc[1:3] # 물가 정보 파싱 관련 테스트 위해 추가
])

In [305]:
summary_df = get_summary_df(test_df)

캐시 없음 → 빈 DF 생성
신규 샘플 발견: 15개
Summarizing [spnews_101404] ...
Job Complete! [spnews_101404] (1/15)
Summarizing [spnews_101403] ...
Job Complete! [spnews_101403] (2/15)
Summarizing [spnews_101402] ...
Job Complete! [spnews_101402] (3/15)
Summarizing [spnews_101394] ...
Job Complete! [spnews_101394] (4/15)
Summarizing [spnews_101392] ...
Job Complete! [spnews_101392] (5/15)
Summarizing [spnews_101391] ...
Job Complete! [spnews_101391] (6/15)
Summarizing [spnews_101390] ...
Job Complete! [spnews_101390] (7/15)
Summarizing [spnews_101389] ...
Job Complete! [spnews_101389] (8/15)
Summarizing [spnews_101383] ...
Job Complete! [spnews_101383] (9/15)
Summarizing [spnews_101377] ...
Job Complete! [spnews_101377] (10/15)
Summarizing [spnews_101376] ...
Job Complete! [spnews_101376] (11/15)
Summarizing [spnews_101374] ...
Job Complete! [spnews_101374] (12/15)
Summarizing [spnews_101373] ...
Job Complete! [spnews_101373] (13/15)
Summarizing [spnews_101146] ...
Job Complete! [spnews_101146] (14/15)

In [307]:
summary_df.to_excel('data/test_llm_summary_1119.xlsx', index=False)

## ISSUE (11.19) - None 타입 반환 에러 (해결 완료)

In [181]:
df = pd.read_csv('data/spnews_ver2.csv')
df = df[df.contents.notnull() & df.category.notnull()]
test_df = df[df.id == 'spnews_101391']

In [182]:
summary_df = get_article_summary(test_df)

Summarizing [spnews_101391] ...
Parsing error: {
    "summary": "북한 평안남도 원화농장에서 첫 모내기를 기념하여 결산분배가 진행되었다. 농장 일꾼들과 근로자들의 노력으로 국가알곡생산계획이 127.5% 달성되었다. 결산분배모임에는 평안남도당위원회 책임비서와 도농촌경리위원회 위원장이 참석했다.",
    "event_title": "첫 모내기 결산분배",
    "event_date": "2025-11-16",
    "event_person": "리경철, 최영송",
    "event_org": "평안남도당위원회, 도농촌경리위원회",
    "event_loc": "북한, 평안남도, 원화농장",
    "keywords": "리경철, 최영송, 평안남도, 원화농장, 결산분배, 첫 모내기",
    "p_rice(won/kg)": null,
    "p_corn(won/kg)": null,
    "p_usd(won/usd)": null
}


TypeError: cannot unpack non-iterable NoneType object

In [193]:
title = test_df.iloc[0].title
contents = test_df.iloc[0].contents
publish_date = test_df.iloc[0].publish_date

In [191]:
# result, input_tokens, output_tokens = generate_event_info(title, contents, publish_date)

Parsing error: {
    "summary": "북한 평안남도 원화농장에서 첫 모내기를 기념하여 결산분배가 진행됐다. 농장 일꾼들과 근로자들의 노력으로 국가알곡생산계획이 127.5% 달성되었다. 결산분배모임에는 평안남도당위원회 책임비서와 도농촌경리위원회 위원장이 참석했다.",
    "event_title": "첫 모내기 결산분배",
    "event_date": "2025-05-16",
    "event_person": "리경철, 최영송",
    "event_org": "평안남도당위원회, 도농촌경리위원회",
    "event_loc": "북한, 평안남도",
    "keywords": "리경철, 최영송, 평안남도, 첫 모내기, 결산분배",
    "p_rice(won/kg)": null,
    "p_corn(won/kg)": null,
    "p_usd(won/usd)": null
}


TypeError: cannot unpack non-iterable NoneType object

In [194]:
prompt = f"""
아래 기사를 분석하여 요구된 정보를 작성하시오.

# 기사 제목:
{title}

# 기사 내용:
{contents}

# 기사 작성일:
{publish_date}

1. 아래 형식으로 정리 (괄호안 각 key값의 한글 설명은 참고만 하고 최종 결과에는 포함하지 않음)
- summary(주요 사건 요약):
- event_title(사건 주제):
- event_date(사건 발생일):
- event_person(사건 핵심 인물):
- event_org(사건 핵심 조직/기관):
- event_loc(사건 발생 지명):
- keywords(주요 키워드):
- p_rice(won/kg)(쌀 가격):
- p_corn(won/kg)(옥수수 가격):
- p_usd(won/usd)(달러 가격)

2. 각 카테고리의 조건
- "주요 사건 요약"은 3 문장 이하로 핵심 내용만 발췌
- "사건 발생일" 과 "기사 작성일"은 yyyy-mm-dd 형식, 
- "사건 발생일"이 명시되지 않았으면 "기사 내용" 중 시간 또는 기간을 나타내는 단어(예시로, '어제', '사흘전', '일주일 전' 등)를 참고하여 "기사 작성일" 기준 계산
- "사건 핵심 인물"은 뉴스의 중심 인물(들)의 이름만을 입력, 다수의 경우 쉼표로 구분
- "사건 핵심 조직/기관"은 뉴스에서 중요하게 다루고 있는 조직 및 기관의 이름만 입력, 다수면 쉼표로 구분, 해당 뉴스의 출처 정보는 제외(예, 노동신문 등).
- "사건 발생 지명"은 [국가, 도, 시]단위 지명만을 입력, 건물등에서 일어난 사건의 경우는 해당 장소의 [국가, 도, 시] 지명을 입력.
- "주요 키워드"는 "사건 핵심 인물", "사건 핵심 조직/기관", "사건 발생 지명"을 반드시 포함하고, "주요 사건 요약" 정보를 가장 잘 묘사하는 최소의 단어들로 쉼표 구분하여 임력.
- 물가 관련 뉴스에 쌀, 옥수수 kg당 가격 및 원달러 환율이 제공될 경우, 반드시 평양의 정보만 선택하여 각 값을 정수로 입력. 
- 위 결과를 종합하여 딕셔너리 형태로 출력
- 설명 금지, 답만 출력
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "당신은 북한 관련 뉴스 사건 정보를 추출하는 전문 분석 모델입니다."},
        {"role": "user", "content": prompt},
    ],
    temperature=0
)

# 소모 토큰량 추출
input_tokens = response.usage.prompt_tokens
output_tokens = response.usage.completion_tokens


result_text = response.choices[0].message.content.strip()

In [197]:
result_text

'{\n    "summary": "북한 평안남도 원화농장에서 첫 모내기를 기념하여 결산분배가 진행됐다. 농장 일꾼들과 근로자들의 노력으로 국가 알곡 생산 계획이 127.5% 달성되었다. 결산분배 모임에는 평안남도당위원회 책임비서와 도농촌경리위원회 위원장이 참석했다.",\n    "event_title": "첫 모내기 결산분배",\n    "event_date": "2025-11-16",\n    "event_person": "리경철, 최영송",\n    "event_org": "평안남도당위원회, 도농촌경리위원회",\n    "event_loc": "북한, 평안남도, 원화농장",\n    "keywords": "리경철, 최영송, 평안남도당위원회, 도농촌경리위원회, 북한, 원화농장, 첫 모내기",\n    "p_rice(won/kg)": null,\n    "p_corn(won/kg)": null,\n    "p_usd(won/usd)": null\n}'

In [203]:
result = json.loads(result_text)  # dict로 파싱 (구조가 명확하므로 안전)
result

{'summary': '북한 평안남도 원화농장에서 첫 모내기를 기념하여 결산분배가 진행됐다. 농장 일꾼들과 근로자들의 노력으로 국가 알곡 생산 계획이 127.5% 달성되었다. 결산분배 모임에는 평안남도당위원회 책임비서와 도농촌경리위원회 위원장이 참석했다.',
 'event_title': '첫 모내기 결산분배',
 'event_date': '2025-11-16',
 'event_person': '리경철, 최영송',
 'event_org': '평안남도당위원회, 도농촌경리위원회',
 'event_loc': '북한, 평안남도, 원화농장',
 'keywords': '리경철, 최영송, 평안남도당위원회, 도농촌경리위원회, 북한, 원화농장, 첫 모내기',
 'p_rice(won/kg)': None,
 'p_corn(won/kg)': None,
 'p_usd(won/usd)': None}

-  Json으로 파싱하면 잘 뽑힘

In [208]:
# 수정된 generate_event_info() 사용
result, input_tokens, output_tokens = generate_event_info(title, contents, publish_date)

In [209]:
result

{'summary': '북한 평안남도 원화농장에서 첫 모내기를 기념하여 결산분배가 진행됐다. 농장 일꾼들과 근로자들의 노력으로 국가알곡생산계획이 127.5% 달성되었다. 결산분배모임에는 평안남도당위원회 책임비서와 도농촌경리위원회 위원장이 참석했다.',
 'event_title': '첫 모내기 결산분배',
 'event_date': '2025-05-16',
 'event_person': '리경철, 최영송',
 'event_org': '평안남도당위원회, 도농촌경리위원회',
 'event_loc': '북한, 평안남도',
 'keywords': '리경철, 최영송, 평안남도, 첫 모내기, 결산분배',
 'p_rice(won/kg)': None,
 'p_corn(won/kg)': None,
 'p_usd(won/usd)': None}

## 테스트 (11.19) - LLM 요약 함수 캐시 기능 테스트

In [248]:
df = pd.read_csv('data/spnews_ver2.csv')

test_df = pd.concat([
    df.head(13),
    df[df.title.str.contains('물가')].iloc[1:3] # 물가 정보 파싱 관련 테스트 위해 추가
])

In [250]:
new_cols = ["summary", "keywords", "event_title", "event_date", "event_person",
            "event_org", "event_loc", "p_rice(won/kg)", "p_corn(won/kg)",
            "p_usd(won/usd)", "job_cost"]

# 캐시가 있으면 로드, 없으면 빈 DataFrame 생성
if os.path.exists(cache_path):
    print("⏳ 캐시 파일 로드 중...")
    output_df = pd.read_csv(cache_path)
    print("✔ 캐시 반영 완료!")
else:
    output_df = pd.DataFrame(columns=(["id"]+new_cols))

In [254]:
 # 캐시에 이미 존재하는 id 목록
cached_ids = set(output_df["id"].tolist())

# 새 df에서 캐시에 없던 id들만 처리
new_rows = test_df[~test_df["id"].isin(cached_ids)]

---

## 테스트 (11.18) - 토큰 비용 계산

In [167]:
# 테스트df 복사본 생성
test_data = test_df.copy()

# 테스트 데이터의 게재일 datetime으로 변환
test_data['publish_date'] = pd.to_datetime(test_data['publish_date']).dt.date
test_data.head(2)

# 테스트 데이터 저장
os.makedirs('data', exist_ok=True)
test_data.to_csv('data/test_dataset.csv', index=False)


In [168]:
test_item = test_data.iloc[-1]
title, contents, publish_date = test_item['title'], test_item['contents'], test_item['publish_date']

print(title, contents, publish_date, sep='\n')

[北 물가] 가을 추수 영향, 곡물가 내림세
북한지역에서 가을 추수가 마무리 되면서 쌀과 옥수수 등 곡물류 가격이 내림세로 돌아섰다
'SPN'이 11월 15일 기준 북한 평양시와 양강도 혜산시, 강원도 원산시, 황해북도 사리원시 장마당 물가를 조사한 결과 이같이 나타났다.
달러는 평양 34,600원(+400원), 혜산 34,630원(+330원), 원산 34,580원(-1,600원), 사리원 34,600원(-1,600원)으로 오르고 내렸다.
쌀(1kg)은 평양 19,850원(보름전보다 -1,150원), 혜산 20,000원(-1,400원), 원산 19,700원(-1,200원), 사리원  19,800원(-1,150원)으로 내림세를 보였다.
옥수수(1kg)는 평양 4,500원(-500원), 혜산 4,500원(-350원), 원산 4,300원(-600원), 사리원 4,600원(-350원)으로 하락했다.
현지 소식통들은 "가을 추수가 마무리되면서 출하량이  늘어나 곡물류 가격이 내리고 있다"고 전했다.
기초식품인 식용유는 달러화가 소폭 상승하거나 내리면서 가격이 들쭉날쭉했다.
식용유(1kg)는 평양 55,800원(-350원), 혜산 55,400원(+300원), 원산 56,000원(-500원)로 조사됐다. 설탕은 평양 40,900원(-600원), 혜산 40,600원(-600원)에 거래됐다.
휘발유(1kg)는 평양 41,100원(-700원), 혜산 41,400원(-600원)으로 내림세를 보였다. 경유(1kg)는 평양 39,400원(-900원), 혜산 39,700원(-1,100원)으로 소폭 내렸다.
소식통들은 "유류가 하락은 가을 추수가 마무리 되면서 수요가 감소하고 공급량이 증가한 것이 원인"이라고 밝혔다.
돼지고기(1kg)는 평양 70,000원(0원), 혜산 71,000원(+200원)에 거래됐다.
2025-11-17


In [169]:
prompt = f"""
아래 기사를 분석하여 요구된 정보를 작성하시오.

# 기사 제목:
{title}

# 기사 내용:
{contents}

# 기사 작성일:
{publish_date}

1. 아래 형식으로 정리
- 주요 사건 요약(summary):
- 사건 주제(event_title):
- 사건 발생일(event_date):
- 사건 발생 핵심 인물(event_person):
- 사건 관련 핵심 조직(event_org):
- 사건 발생 지명(event_loc):
- 주요 키워드(keywords):

2. 각 카테고리의 조건
- "주요 사건 요약"은 3문장 이하로 핵심 내용만 발췌
- "사건 발생일" 과 "기사 작성일"은 yyyy-mm-dd 형식, 
- 사건 발생일"이 명시되지 않았으면 "기사 내용" 중 시간 또는 기간을 나타내는 단어(예시로, '어제', '사흘전', '일주일 전' 등)를 참고하여 "기사 작성일" 기준 계산
- "사건 발생 핵심 인물"은 뉴스의 주체가 되는 주요 인물명만, 다수면 쉼표로 구분
- "사건 관련 핵심 조직"은 뉴스에서 중요하게 다루고 있는 단체 및 기관명만, 다수면 쉼표로 구분, 해당 뉴스의 출처에 해당하는 정보는 제외(예시로, 노동신문 등).
- "사건 발생 지명"은 사건이 발생한 지역 또는 사건의 주체에 해당하는 단 하나의 지명만 입력
- "주요 키워드"는 뉴스를 가장 잘 표현할 수 있는 단어들로만 구성, 다섯개 까지, '북한' 이라는 단어는 제외
- 출력은 딕셔너리 형태로, 값이 여러 개면 리스트로 묶어서 출력
- 설명 금지, 답만 출력
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "당신은 북한 관련 뉴스 사건 정보를 추출하는 전문 분석 모델입니다."},
        {"role": "user", "content": prompt},
    ],
    temperature=0
)

# 소모 토큰량 추출
input_tokens = response.usage.prompt_tokens
output_tokens = response.usage.completion_tokens


result_text = response.choices[0].message.content.strip()

# 문자열을 dict로 변환
try:
    result = eval(result_text)  # dict로 파싱 (구조가 명확하므로 안전)
except:
    print("Parsing error:", result_text)

In [170]:
result

{'주요 사건 요약': '북한에서 가을 추수가 마무리되면서 쌀과 옥수수 등 곡물류 가격이 하락세를 보이고 있다. 평양, 혜산, 원산, 사리원 등지에서 곡물 가격이 내림세로 돌아섰으며, 유류 가격도 하락했다. 현지 소식통은 출하량 증가가 가격 하락의 원인이라고 전했다.',
 '사건 주제': '곡물가 하락',
 '사건 발생일': '2025-11-15',
 '사건 발생 핵심 인물': '',
 '사건 관련 핵심 조직': '',
 '사건 발생 지명': '평양',
 '주요 키워드': ['곡물', '가격', '추수', '하락', '유류']}

In [ ]:
# 프롬프트 변경 후(6차 변경 프롬프트 사용)
result, input_tokens, output_tokens = generate_event_info(title, contents, publish_date)
result

In [174]:
# 토큰 사용량 조회
print(f'***토큰 사용량***\n입력: {input_tokens}\n출력: {output_tokens}')
print(f'-'*30)

# 전체 소비 토큰 가격 계산
costs = sum([(input_tokens * 0.15 / 1_000_000), (output_tokens * 0.60 / 1_000_000)]) # 가독성을 위해 1000단위 underscore(_) 사용
print(f'***Total Cost***\n${costs}')


***토큰 사용량***
입력: 1085
출력: 175
------------------------------
***Total Cost***
$0.00026775


## 테스트 (11.18) - 뉴스 요약 테스트 (임의의 16개 뉴스 샘플)

In [108]:
df = pd.read_csv('data/test_dataset.csv')
print(len(df))
df.head()

16


,id,title,contents,author,publish_date,url,category
0,spnews_101049,"北, 김영남 상임위원장 영결식...김정은 참석",고(故) 김영남 북한 최고인민회의 상임위원회 위원장 장례식이 5일 평양에서 국장으로...,안윤석 대기자,2025-11-06,https://www.spnews.co.kr/news/articleView.html...,정치
1,spnews_101366,"박태성 北 내각총리, 광산·발전소 등 경제 여러 부문 현장 확인",박태성 북한 내각총리가 광산과 발전소 등 경제 여러 부문 현장을 확인했다.\n노동신...,안윤석 대기자,2025-11-16,https://www.spnews.co.kr/news/articleView.html...,정치
2,spnews_100345,"김정은, '신의주온실종합농장' 또 현지지도...""농촌문명의 새로운 경지 개척""",김정은 북한 총비서가 17일 마감단계에 들어선 신의주온실종합농장 건설장을 또다시 방...,안윤석 대기자,2025-10-18,https://www.spnews.co.kr/news/articleView.html...,정치
3,spnews_101306,"박태성 北 내각총리, 통싸완 폼비한 라오스 외무상 접견",박태성 북한 내각총리가 13일 만수대의사당에서 통싸완 폼비한 라오스 외무상을 만났다...,안윤석 대기자,2025-11-14,https://www.spnews.co.kr/news/articleView.html...,외교
4,spnews_100889,"북한-러시아, 무역경제 및 과학기술협력 추진 회담...의정서 조인",북한과 러시아가 합의된 다방면적인 쌍무협력계획 이행을 추진하기 위한 회담을 31일 ...,안윤석 대기자,2025-11-01,https://www.spnews.co.kr/news/articleView.html...,외교


In [ ]:
# 새 컬럼 추가 (job_cost 컬럼은 임시 컬럼)
new_cols = ["summary", "keywords", "event_title", "event_date", "event_obj", "event_loc", "job_cost"]
for col in new_cols:
    df[col] = None

# 각 row 반복 → LLM 호출 → 결과 입력
for idx, row in df.iterrows():
    print(f"Processing index {idx} ...")

    title = row["title"]
    contents = row["contents"]
    publish_date = row["publish_date"]  # datetime.date

    result, input_tokens, output_tokens = generate_event_info(title, contents, publish_date)

    if result:
        df.at[idx, "summary"] = result.get("summary")
        df.at[idx, "keywords"] = result.get("keywords")
        df.at[idx, "event_title"] = result.get("event_title")
        df.at[idx, "event_date"] = result.get("event_date")
        df.at[idx, "event_obj"] = result.get("event_obj")
        df.at[idx, "event_loc"] = result.get("event_loc")
        
        # LLM 토큰 비용 컬럼 추가(테스트용)
        df.at[idx, "job_cost"] = sum([(input_tokens * 0.15 / 1_000_000), (output_tokens * 0.60 / 1_000_000)])
    
    print(f'Work Finish ({idx+1}/{len(df)})')
print("전체 작업 완료!")

Processing index 0 ...
Work Finish (1/16)
Processing index 1 ...
Work Finish (2/16)
Processing index 2 ...
Work Finish (3/16)
Processing index 3 ...
Work Finish (4/16)
Processing index 4 ...
Work Finish (5/16)
Processing index 5 ...
Work Finish (6/16)
Processing index 6 ...
Work Finish (7/16)
Processing index 7 ...
Work Finish (8/16)
Processing index 8 ...
Work Finish (9/16)
Processing index 9 ...
Work Finish (10/16)
Processing index 10 ...
Work Finish (11/16)
Processing index 11 ...
Work Finish (12/16)
Processing index 12 ...
Work Finish (13/16)
Processing index 13 ...
Work Finish (14/16)
Processing index 14 ...
Work Finish (15/16)
Processing index 15 ...
Work Finish (16/16)
전체 작업 완료!


### --- Issue 발생--- 요약 정보 추출 실패 하는 경우 발생

In [112]:
df

,id,title,contents,author,publish_date,url,category,summary,keywords,event_title,event_date,event_obj,event_loc,job_cost
0,spnews_101049,"北, 김영남 상임위원장 영결식...김정은 참석",고(故) 김영남 북한 최고인민회의 상임위원회 위원장 장례식이 5일 평양에서 국장으로...,안윤석 대기자,2025-11-06,https://www.spnews.co.kr/news/articleView.html...,정치,None,None,None,None,None,None,0.000259
1,spnews_101366,"박태성 北 내각총리, 광산·발전소 등 경제 여러 부문 현장 확인",박태성 북한 내각총리가 광산과 발전소 등 경제 여러 부문 현장을 확인했다.\n노동신...,안윤석 대기자,2025-11-16,https://www.spnews.co.kr/news/articleView.html...,정치,None,None,None,None,None,None,0.000253
2,spnews_100345,"김정은, '신의주온실종합농장' 또 현지지도...""농촌문명의 새로운 경지 개척""",김정은 북한 총비서가 17일 마감단계에 들어선 신의주온실종합농장 건설장을 또다시 방...,안윤석 대기자,2025-10-18,https://www.spnews.co.kr/news/articleView.html...,정치,None,None,None,None,None,None,0.00027
3,spnews_101306,"박태성 北 내각총리, 통싸완 폼비한 라오스 외무상 접견",박태성 북한 내각총리가 13일 만수대의사당에서 통싸완 폼비한 라오스 외무상을 만났다...,안윤석 대기자,2025-11-14,https://www.spnews.co.kr/news/articleView.html...,외교,None,None,None,None,None,None,0.000208
4,spnews_100889,"북한-러시아, 무역경제 및 과학기술협력 추진 회담...의정서 조인",북한과 러시아가 합의된 다방면적인 쌍무협력계획 이행을 추진하기 위한 회담을 31일 ...,안윤석 대기자,2025-11-01,https://www.spnews.co.kr/news/articleView.html...,외교,None,None,None,None,None,None,0.000258
5,spnews_100318,북러 과학기술협조위 임업분과위 회의...벌목공 송출 논의?,북한과 러시아 무역경제 및 과학기술협조위원회 임업분과위원회 제28차회의가 16일 평...,안윤석 대기자,2025-10-17,https://www.spnews.co.kr/news/articleView.html...,외교,None,None,None,None,None,None,0.000245
6,spnews_101377,"北, 한미 해군 연합훈련 기간 강원도 고성항에 호위함 추가 배치",북한이 최근 진행된 한미 해군 연합훈련 기간 강원도 고성항에 두만급(1천500t급)...,유영목 기자,2025-11-16,https://www.spnews.co.kr/news/articleView.html...,군사,None,None,None,None,None,None,0.000231
7,spnews_101097,"北, 동해로 탄도미사일 발사...美제재에 '반발 담화' 하루 만에 도발(종합)","북한이 16일 만에 탄도미사일을 또다시 발사했다.\n합동참모본보는 ""우리 군은 7일...",유영목 기자,2025-11-07,https://www.spnews.co.kr/news/articleView.html...,군사,북한이 16일 만에 동해로 단거리 탄도미사일을 발사했다. 이번 발사는 미국의 대북 ...,"[북한, 탄도미사일, 대북 제재, 동해, 미국]",북한 탄도미사일 발사,2025-11-07,None,북한 평북 대관,0.00025
8,spnews_100356,"북한, 일본 자위대 구축함 미국산 '장거리순항미사일' 탑재 비난",북한이 일본 자위대 구축함의 미국산 토마호크 장거리순항미사일 탑재를 비난했다.\n북...,안윤석 대기자,2025-10-18,https://www.spnews.co.kr/news/articleView.html...,군사,None,None,None,None,None,None,0.000269
9,spnews_101404,"北, 신의주온실종합농장 건설장 지대정리·잔디심기 마감단계",북한 평안북도 신의주온실종합농장 건설장에서 방대한 면적의 지대정리와 잔디심기가 마감...,안윤석 대기자,2025-11-17,https://www.spnews.co.kr/news/articleView.html...,경제/산업,None,None,None,None,None,None,0.000237


##### Idx 0번 뉴스 하나만 가지고 테스트

In [115]:
df = pd.read_csv('data/test_dataset.csv')
sample_data = df.iloc[[0]]
sample_data

,id,title,contents,author,publish_date,url,category
0,spnews_101049,"北, 김영남 상임위원장 영결식...김정은 참석",고(故) 김영남 북한 최고인민회의 상임위원회 위원장 장례식이 5일 평양에서 국장으로...,안윤석 대기자,2025-11-06,https://www.spnews.co.kr/news/articleView.html...,정치


In [116]:
title = sample_data['title']
contents = sample_data['contents']
publish_date = sample_data['publish_date']

print(title, contents, publish_date, sep='\n')

0    北, 김영남 상임위원장 영결식...김정은 참석
Name: title, dtype: object
0    고(故) 김영남 북한 최고인민회의 상임위원회 위원장 장례식이 5일 평양에서 국장으로...
Name: contents, dtype: object
0    2025-11-06
Name: publish_date, dtype: object


In [124]:
result, input_tokens, output_tokens = generate_event_info(title, contents, publish_date)

In [125]:
result

{'summary': '고(故) 김영남 북한 최고인민회의 상임위원회 위원장 장례식이 5일 평양에서 국장으로 진행되었다. 김정은이 참석하여 고인의 마지막 길을 배웅했다. 장례식은 북한의 공식적인 절차에 따라 치러졌다.',
 'event_title': '김영남 상임위원장 영결식',
 'event_date': '2025-11-05',
 'event_person': '김영남, 김정은',
 'event_org': '북한 최고인민회의',
 'event_loc': '평양',
 'keywords': ['김영남', '영결식', '김정은', '북한', '국장']}

###### 문제 해결: 컬럼명 불일치 문제 - event_person추가필요, event_org로 수정 필요 

- 컬럼 구성 및 순서 정리 

new_cols = ["summary", "keywords", "event_title", "event_date", "event_person", "event_org", "event_loc", "job_cost"]

### --- Issue 해결 --- 수정된 테스트 코드 (5개의 샘플만 테스트)

In [154]:
df = pd.read_csv('data/test_dataset.csv')
print(len(df))
df

16


,id,title,contents,author,publish_date,url,category
0,spnews_101049,"北, 김영남 상임위원장 영결식...김정은 참석",고(故) 김영남 북한 최고인민회의 상임위원회 위원장 장례식이 5일 평양에서 국장으로...,안윤석 대기자,2025-11-06,https://www.spnews.co.kr/news/articleView.html...,정치
1,spnews_101366,"박태성 北 내각총리, 광산·발전소 등 경제 여러 부문 현장 확인",박태성 북한 내각총리가 광산과 발전소 등 경제 여러 부문 현장을 확인했다.\n노동신...,안윤석 대기자,2025-11-16,https://www.spnews.co.kr/news/articleView.html...,정치
2,spnews_100345,"김정은, '신의주온실종합농장' 또 현지지도...""농촌문명의 새로운 경지 개척""",김정은 북한 총비서가 17일 마감단계에 들어선 신의주온실종합농장 건설장을 또다시 방...,안윤석 대기자,2025-10-18,https://www.spnews.co.kr/news/articleView.html...,정치
3,spnews_101306,"박태성 北 내각총리, 통싸완 폼비한 라오스 외무상 접견",박태성 북한 내각총리가 13일 만수대의사당에서 통싸완 폼비한 라오스 외무상을 만났다...,안윤석 대기자,2025-11-14,https://www.spnews.co.kr/news/articleView.html...,외교
4,spnews_100889,"북한-러시아, 무역경제 및 과학기술협력 추진 회담...의정서 조인",북한과 러시아가 합의된 다방면적인 쌍무협력계획 이행을 추진하기 위한 회담을 31일 ...,안윤석 대기자,2025-11-01,https://www.spnews.co.kr/news/articleView.html...,외교
5,spnews_100318,북러 과학기술협조위 임업분과위 회의...벌목공 송출 논의?,북한과 러시아 무역경제 및 과학기술협조위원회 임업분과위원회 제28차회의가 16일 평...,안윤석 대기자,2025-10-17,https://www.spnews.co.kr/news/articleView.html...,외교
6,spnews_101377,"北, 한미 해군 연합훈련 기간 강원도 고성항에 호위함 추가 배치",북한이 최근 진행된 한미 해군 연합훈련 기간 강원도 고성항에 두만급(1천500t급)...,유영목 기자,2025-11-16,https://www.spnews.co.kr/news/articleView.html...,군사
7,spnews_101097,"北, 동해로 탄도미사일 발사...美제재에 '반발 담화' 하루 만에 도발(종합)","북한이 16일 만에 탄도미사일을 또다시 발사했다.\n합동참모본보는 ""우리 군은 7일...",유영목 기자,2025-11-07,https://www.spnews.co.kr/news/articleView.html...,군사
8,spnews_100356,"북한, 일본 자위대 구축함 미국산 '장거리순항미사일' 탑재 비난",북한이 일본 자위대 구축함의 미국산 토마호크 장거리순항미사일 탑재를 비난했다.\n북...,안윤석 대기자,2025-10-18,https://www.spnews.co.kr/news/articleView.html...,군사
9,spnews_101404,"北, 신의주온실종합농장 건설장 지대정리·잔디심기 마감단계",북한 평안북도 신의주온실종합농장 건설장에서 방대한 면적의 지대정리와 잔디심기가 마감...,안윤석 대기자,2025-11-17,https://www.spnews.co.kr/news/articleView.html...,경제/산업


In [155]:
# 새 컬럼 추가 (job_cost 컬럼은 임시 컬럼)
new_cols = ["summary", "keywords", "event_title", "event_date", "event_person", "event_org", "event_loc", "job_cost"] # 파싱 문제 일으킨 col명 리스트 수정
for col in new_cols:
    df[col] = None

# 각 row 반복 → LLM 호출 → 결과 입력
for idx, row in df.iterrows():
    print(f"Processing index {idx} ...")

    title = row["title"]
    contents = row["contents"]
    publish_date = row["publish_date"]  # datetime.date

    result, input_tokens, output_tokens = generate_event_info(title, contents, publish_date)

    if result:
        df.at[idx, "summary"] = result.get("summary")
        df.at[idx, "keywords"] = result.get("keywords")
        df.at[idx, "event_title"] = result.get("event_title")
        df.at[idx, "event_date"] = result.get("event_date")
        df.at[idx, "event_person"] = result.get("event_person")
        df.at[idx, "event_org"] = result.get("event_org")
        df.at[idx, "event_loc"] = result.get("event_loc")
        
        # LLM 토큰 비용 컬럼 추가(테스트용)
        df.at[idx, "job_cost"] = sum([(input_tokens * 0.15 / 1_000_000), (output_tokens * 0.60 / 1_000_000)])
    
    print(f'Work Finish ({idx+1}/{len(df)})')
print("전체 작업 완료!")

Processing index 0 ...
Work Finish (1/16)
Processing index 1 ...
Work Finish (2/16)
Processing index 2 ...
Work Finish (3/16)
Processing index 3 ...
Work Finish (4/16)
Processing index 4 ...
Work Finish (5/16)
Processing index 5 ...
Work Finish (6/16)
Processing index 6 ...
Work Finish (7/16)
Processing index 7 ...
Work Finish (8/16)
Processing index 8 ...
Work Finish (9/16)
Processing index 9 ...
Work Finish (10/16)
Processing index 10 ...
Work Finish (11/16)
Processing index 11 ...
Work Finish (12/16)
Processing index 12 ...
Work Finish (13/16)
Processing index 13 ...
Work Finish (14/16)
Processing index 14 ...
Work Finish (15/16)
Processing index 15 ...
Work Finish (16/16)
전체 작업 완료!


In [156]:
df

,id,title,contents,author,publish_date,url,category,summary,keywords,event_title,event_date,event_person,event_org,event_loc,job_cost
0,spnews_101049,"北, 김영남 상임위원장 영결식...김정은 참석",고(故) 김영남 북한 최고인민회의 상임위원회 위원장 장례식이 5일 평양에서 국장으로...,안윤석 대기자,2025-11-06,https://www.spnews.co.kr/news/articleView.html...,정치,김영남 북한 최고인민회의 상임위원장 장례식이 평양에서 국장으로 치러졌다. 김정은 총...,"김영남, 영결식, 김정은, 평양, 애국열사릉",김영남 상임위원장 영결식,2025-11-05,"김정은, 박태성, 김영남","당 정치국, 국가장의위원회, 최고인민회의 상임위원회, 내각",평양,0.000251
1,spnews_101366,"박태성 北 내각총리, 광산·발전소 등 경제 여러 부문 현장 확인",박태성 북한 내각총리가 광산과 발전소 등 경제 여러 부문 현장을 확인했다.\n노동신...,안윤석 대기자,2025-11-16,https://www.spnews.co.kr/news/articleView.html...,정치,박태성 북한 내각총리가 광산과 발전소 등 경제 여러 부문 현장을 확인했다. 그는 석...,"박태성, 광산, 발전소, 석탄 생산, 비료 생산","박태성 北 내각총리, 광산·발전소 등 경제 여러 부문 현장 확인",2025-11-16,박태성,"북한 내각, 석탄공업성, 남흥청년화학연합기업소",북한,0.000254
2,spnews_100345,"김정은, '신의주온실종합농장' 또 현지지도...""농촌문명의 새로운 경지 개척""",김정은 북한 총비서가 17일 마감단계에 들어선 신의주온실종합농장 건설장을 또다시 방...,안윤석 대기자,2025-10-18,https://www.spnews.co.kr/news/articleView.html...,정치,김정은 총비서가 신의주온실종합농장 건설장을 방문하여 진행 상황을 점검했다. 그는 농...,"김정은, 신의주온실종합농장, 농촌문명, 건설, 지역경제",신의주온실종합농장 현지지도,2025-10-17,김정은,"군부대, 청년돌격대",신의주,0.000256
3,spnews_101306,"박태성 北 내각총리, 통싸완 폼비한 라오스 외무상 접견",박태성 북한 내각총리가 13일 만수대의사당에서 통싸완 폼비한 라오스 외무상을 만났다...,안윤석 대기자,2025-11-14,https://www.spnews.co.kr/news/articleView.html...,외교,박태성 북한 내각총리가 라오스 외무상 통싸완 폼비한을 만났다. 이 자리에는 외무성 ...,"북한, 라오스, 외무상, 접견, 참배","박태성 북한 내각총리, 라오스 외무상 접견",2025-11-13,"박태성, 통싸완 폼비한","북한 내각, 라오스 외무부",만수대의사당,0.000199
4,spnews_100889,"북한-러시아, 무역경제 및 과학기술협력 추진 회담...의정서 조인",북한과 러시아가 합의된 다방면적인 쌍무협력계획 이행을 추진하기 위한 회담을 31일 ...,안윤석 대기자,2025-11-01,https://www.spnews.co.kr/news/articleView.html...,외교,북한과 러시아가 무역경제 및 과학기술협력 강화를 위한 회담을 평양에서 개최했다. 회...,"북한, 러시아, 무역경제, 과학기술협력, 의정서",북한-러시아 무역경제 및 과학기술협력 회담,2025-10-31,"윤정호, 알렉산드르 코즐로프, 김덕훈","북러 정부간 무역경제 및 과학기술협조위원회, 북한 대외경제성",평양,0.000259
5,spnews_100318,북러 과학기술협조위 임업분과위 회의...벌목공 송출 논의?,북한과 러시아 무역경제 및 과학기술협조위원회 임업분과위원회 제28차회의가 16일 평...,안윤석 대기자,2025-10-17,https://www.spnews.co.kr/news/articleView.html...,외교,북한과 러시아의 임업분과위원회 제28차 회의가 평양에서 열렸다. 회의에서는 임업 분...,"북러, 임업, 벌목공, 송출, 협조",북러 과학기술협조위 임업분과위 회의,2025-10-16,"한영호, 그리고리 구세프","북러 무역경제 및 과학기술협조위원회, 공업무역성 임업대표단",평양,0.000255
6,spnews_101377,"北, 한미 해군 연합훈련 기간 강원도 고성항에 호위함 추가 배치",북한이 최근 진행된 한미 해군 연합훈련 기간 강원도 고성항에 두만급(1천500t급)...,유영목 기자,2025-11-16,https://www.spnews.co.kr/news/articleView.html...,군사,북한이 한미 해군 연합훈련 기간 동안 강원도 고성항에 두만급 호위함을 추가 배치했다...,"북한, 호위함, 한미 해군, 연합훈련, 고성항",북한 호위함 추가 배치,2025-11-11,,"북한 해군, 한미 해군",고성항,0.000228
7,spnews_101097,"北, 동해로 탄도미사일 발사...美제재에 '반발 담화' 하루 만에 도발(종합)","북한이 16일 만에 탄도미사일을 또다시 발사했다.\n합동참모본보는 ""우리 군은 7일...",유영목 기자,2025-11-07,https://www.spnews.co.kr/news/articleView.html...,군사,북한이 16일 만에 동해로 단거리 탄도미사일을 발사했다. 이번 발사는 미국의 대북 ...,"북한, 탄도미사일, 동해, 대북제재, 이재명 정부",북한 탄도미사일 발사,2025-11-07,,"합동참모본부, 미 재무부, 미 국무부",북한 평북 대관,0.000249
8,spnews_100356,"북한, 일본 자위대 구축함 미국산 '장거리순항미사일' 탑재 비난",북한이 일본 자위대 구축함의 미국산 토마호크 장거리순항미사일 탑재를 비난했다.\n북...,안윤석 대기자,2025-10-18,https://www.spnews.co.kr/news/articleView.html...,군사,북한이 일본 자위대 구축함의 미국산 토마호크 장거리순항미사일 탑재를 비난했다. 북한...,"북한, 일본, 자위대, 토마호크, 군사","북한, 일본 자위대 구축함 미국산 '장거리순항미사일' 탑재 비난",2025-10-18,,"북한, 일본 자위대",일본,0.000246
9,spnews_101404,"北, 신의주온실종합농장 건설장 지대정리·잔디심기 마감단계",북한 평안북도 신의주온실종합농장 건설장에서 방대한 면적의 지대정리와 잔디심기가 마감...,안윤석 대기자,2025-11-17,https://www.spnews.co.kr/news/articleView.html...,경제/산업,북한 신의주온실종합농장에서 지대정리와 잔디심기가 마감단계에 이르고 있다. 군민건설자...,"신의주, 온실농장, 지대정리, 잔디심기, 건설",신의주온실종합농장 건설 진행,2025-11-17,,군민건설자들,신의주,0.000222


In [ ]:
df.to_csv('data/test_dataset_llm.tsv', index=False, sep='\t')

In [185]:
df.to_excel('data/test_dataset_llm.xlsx', index=False)

## 테스트 (11.18) - 사과 농장 뉴스

In [178]:
test_item = ArticleCrawling('https://www.spnews.co.kr/news/articleView.html?idxno=101229').to_dict()

In [182]:
test_item

{'url': 'https://www.spnews.co.kr/news/articleView.html?idxno=101229',
 'id': 'spnews_101229',
 'title': '[북한 경제 단신] 함남 북청군 과일가공공장 개건현대화공사 준공',
 'author': '안윤석 대기자',
 'category': '경제/산업',
 'publish_date': '2025.11.12 07:00',
 'contents': '북한 과수의 고향인 함경남도 북청군에서 과일가공공장 개건현대화공사를 마무리했다.\n노동신문은 12일\xa0\xa0"수만㎡의 부지에 연건축면적이 1만 수천㎡에 이르는 사무청사와 생산건물들을 개건하고 모든 공정이 자동화, 흐름선화된 과일가공품생산기지를 건설했다"고 보도했다.\n"공장에서는\xa0과일농축즙, 과일단물, 과일살즙음료, 말린과일, 과일가루, 과일단묵생산공정을 완공한데 이어 최근 과일통졸임, 과일단졸임, 과일발효식초생산공정을 간춰 총 9가지에 달하는 과일가공품을 생산할 수 있다"고 소개했다.\n신문은 또 "황해북도농업과학연구소에서 지역의 기상기후적 특성에 맞는 두벌농사다수확재배기술을 개발도입했다"고 전했다.\n"연구소에서 올해 생육기일이 짧으면서도 가뭄견딜성이 강하고 불리한 토양조건에서도 수확고가 높은 다수확품종을 육종해 국가품종으로 등록되게 했으며, 재배기술을 확립했다"고 밝혔다.\n"연구소에서는 새로 육종한 다수확품종을 도내 농촌에 확대도입하고 사리원시와 서흥군, 은파군 등 여러 지역에 연구사들을 파견해 재배기술을 보급일반화하는 사업을 진행했다"고 신문은 덧붙였다.'}

In [183]:
result, input_tokens, output_tokens =generate_event_info(test_item['title'], test_item['contents'], pd.to_datetime(test_item['publish_date']).date())
result

{'summary': '함경남도 북청군에서 과일가공공장 개건현대화공사가 완료되었다. 이 공장은 자동화된 과일가공품 생산기지로, 총 9가지의 과일가공품을 생산할 수 있다. 또한, 황해북도농업과학연구소는 지역에 맞는 다수확재배기술을 개발하여 농촌에 보급하고 있다.',
 'event_title': '과일가공공장 개건현대화공사 준공',
 'event_date': '2025-11-12',
 'event_person': '',
 'event_org': '황해북도농업과학연구소',
 'event_loc': '북청군',
 'keywords': '과일가공, 현대화, 자동화, 다수확, 농업기술'}